In [2]:
%pwd

'c:\\Users\\MCW\\Documents\\Ajith\\MLOps\\project_02\\Interstate_Traffic_Volume_Project\\research'

In [3]:
import os 
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\MCW\\Documents\\Ajith\\MLOps\\project_02\\Interstate_Traffic_Volume_Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from trafficVolumePrediction.constants import *
from trafficVolumePrediction.utils.common import read_yaml, create_directories

In [8]:
import os 
import urllib.request as request
import zipfile
from trafficVolumePrediction import logger
from trafficVolumePrediction.utils.common import get_size

In [16]:
import time
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        delay = 5
        max_retries = 3
        for _ in range(max_retries):
            try:
                if not os.path.exists(self.config.local_data_file):
                    filename, headers = request.urlretrieve(
                    url = self.config.source_URL,
                    filename = self.config.local_data_file
                    )
                    logger.info(f"{filename} download! with following info: \n {headers}")
                else:
                    logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            except Exception as e:
                logger.info(f"Delay for next attempt {e}")
                time.sleep(delay)
                delay *= 2
        else:
            logger.info(f"Failed for {self.config.source_URL} after {max_retries} attempt")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )
          return data_ingestion_config

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2023-12-29 19:31:11,607: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-29 19:31:11,609: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-29 19:31:11,610: INFO: common: created directory at: artifacts]
[2023-12-29 19:31:11,612: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-29 19:31:12,504: INFO: 934097324: Delay for next attempt <urlopen error [WinError 10054] An existing connection was forcibly closed by the remote host>]
[2023-12-29 19:31:18,465: INFO: 934097324: Delay for next attempt <urlopen error [WinError 10054] An existing connection was forcibly closed by the remote host>]
[2023-12-29 19:31:29,833: INFO: 934097324: artifacts/data_ingestion/data.zip download! with following info: 
 Connection: close
Content-Length: 671587
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "812acc4135ed854820899400b32cfd7c2570231663e8b